<a href="https://colab.research.google.com/github/v-i-t-o-r/strava/blob/main/marathon_efficiency_chart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Generate efficiency chart for marathon runs
This notebook follows the discussion on [this twitter thread](https://twitter.com/Dan_Nash94/status/1526182083583954945) and calculates your running efficiency for each split of a marathon race.

You need to create a Strava API application in order to use their API. Follow the instructions on this page to create your app: <https://medium.com/@annthurium/getting-started-with-the-strava-api-a-tutorial-f3909496cd2d>

After setting up the app, note down the following information (you will need it to run this notebook):
- Client id
- Client secret

**Note:** Strava imposes some request limits (30'000/day, and 600/every 15min). 

In [ ]:
!pip install stravaio folium

In [ ]:
import os
import logging
import json
import urllib
import requests
import folium
from stravaio import StravaIO

In [ ]:
# Paste your client id and client secret here.
STRAVA_CLIENT_ID = "YOUR_CLIENT_ID"
STRAVA_CLIENT_SECRET = "YOUR_CLIENT_SECRET"

### Authorization with Strava
The cell below creates the proper authorization link using the Stravaio Python library, which is used later to retrieve activities.
It is important to run this cell, just pasting the access_token from your Strava settings will not work, because Stravaio needs to be authorized.

- Run the cell below and click the link that is printed, when prompted click "Authorize" on the website that opens
- After you click "Authorize" you see something like, "This site can't be reached"
- Stay on that page and look at the URL
- The URL will show the authorization code (the bit after "code=" in the URL) and scope you accepted
- Copy the code and paste it below and continue the notebook execution

More detailed info can be found here:
- <https://developers.strava.com/docs/getting-started/>
- <https://developers.strava.com/docs/authentication/>

In [ ]:
params_oauth = {
    "client_id": STRAVA_CLIENT_ID,
    "response_type": "code",
    "redirect_uri": f"http://localhost:8000/authorization_successful",
    "scope": "read,profile:read_all,activity:read",
    "state": 'https://github.com/sladkovm/strava-http', # Sladkovm is the author of the Stravaio library
    "approval_prompt": "force"
}
values_url = urllib.parse.urlencode(params_oauth)
base_url = 'https://www.strava.com/oauth/authorize'
authorize_url = base_url + '?' + values_url
print(authorize_url)

https://www.strava.com/oauth/authorize?client_id=6055&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8000%2Fauthorization_successful&scope=read%2Cprofile%3Aread_all%2Cactivity%3Aread&state=https%3A%2F%2Fgithub.com%2Fsladkovm%2Fstrava-http&approval_prompt=force


In [ ]:
# Paste the code from the URL here. Afterwards there are no manual steps anymore.
AUTHORIZATION_CODE = "YOUR_AUTH_CODE"

In [ ]:
# List of marathon races you want to run efficiency analysis on.
# MARATHONS = {
#     'NAME_1': STRAVA_ID_1,
#     'NAME_2': STRAVA_ID_2,
#     'NAME...': STRAVA_ID...
# }


The following cell retrieves an access token using the authorization code. That access token can then be used to retrieve Strava data.

In [ ]:
payload = {
    "client_id": STRAVA_CLIENT_ID,
    "client_secret": STRAVA_CLIENT_SECRET,
    "grant_type": "authorization_code",
    "code": AUTHORIZATION_CODE,
}

response = requests.request(
    "POST", "https://www.strava.com/api/v3/oauth/token", data=payload
)

response = json.loads(response.text)
TOKEN = response["access_token"]
client = StravaIO(access_token=TOKEN)

In [ ]:
MILE_IN_METERS = 1609.34

def compute_marathon_efficiency_splits(strava_activity_id):
  m_activity = client.activities_api.get_activity_by_id(strava_activity_id)
  m_hr_stream = client.streams_api.get_activity_streams(strava_activity_id, ['heartrate'], key_by_type=False)

  # Strava API doesn't provide AVG HR for each split, so do a rough computation:
  # Get all HR datapoints, check how often (distance) a new datapoint is saved, 
  # compute avg for windows of 1609 metres.
  # Note: this could be improved by looking at the actual measured distance for
  # each stream data point and cross that with split distances.
  n_samples_per_split = (m_hr_stream.heartrate.original_size / m_activity.distance) * MILE_IN_METERS
  hr_splits = {}
  for i in range(1,27):
    start = int(i*n_samples_per_split)
    end = int((i+1)*n_samples_per_split)
    hr_data = m_hr_stream.heartrate.data[start:end]
    hr_splits[i] = (sum(hr_data) / len(hr_data))

  # Get elapsed time for each split, so we can compute total number of beats per
  # split.
  # Note: split distance is actually not 1609 most times, so minor precision
  # improvements can be achieved by using split.distance instead of constant
  # MILE_IN_METERS
  splits_elapsed = {}
  for split in m_activity.splits_standard:
    splits_elapsed[split.split] = split.elapsed_time

  # Compute marathon efficiency splits, aka, meters per beat
  efficiency_splits = {split: MILE_IN_METERS / (avg_hr / 60 * splits_elapsed[split]) for (split, avg_hr) in hr_splits.items()}
  return efficiency_splits


In [ ]:
marathon_efficiency_splits = {m_name : compute_marathon_efficiency_splits(strava_activity_id) for (m_name, strava_activity_id) in MARATHONS.items()}

In [ ]:
# Render the efficiency chart
import matplotlib.pylab as plt

for (mname, ef_splits) in marathon_efficiency_splits.items():
  lists = sorted(ef_splits.items()) # sorted by key, return a list of tuples

  x, y = zip(*lists) # unpack a list of pairs into two tuples

  plt.plot(x, y, label=mname)
plt.legend()  
plt.rcParams['figure.figsize'] = [100, 10]
plt.show()